In [1]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
!pip install gradio

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
INFO: pip is looking at multiple versions of requests to determine which version is compatible with other requirements. This could take a while.
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 28.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 31.5 MB/s eta 0:00:00 0:00:01
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
    opencv-python (>=3.) ; extra == 'all'
                  ~~~~^
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.4
    Uninstalling urllib3-1.26.4:
      Successfully uninstalled urllib3-1.26.4
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typ

In [2]:
# v3 - reactive?

In [4]:
import paho.mqtt.client as mqtt
import json
import time
import uuid
import gradio as gr
import threading

# MQTT broker details
MQTT_BROKER = "recoil.ise.utk.edu"
MQTT_USER = "hivemquser"
MQTT_PWD = "mqAccess2024REC"
MQTT_PORT = 1883

# Global variables
broker_connection_status = False
latest_payload = {}

# Callback function when the client connects to the broker
def on_connect(client, userdata, flags, rc):
    global broker_connection_status
    if rc == 0:
        broker_connection_status = True
        print("Connected successfully")
        # client.subscribe("json_data")  # Ensure subscription happens upon connection
        client.subscribe("#") # all topics
    else:
        print(f"Connection failed with code {rc}")

# Callback function when a message is received from the broker
def on_message(client, userdata, message):
    global latest_payload
    try:
        payload = message.payload.decode("utf-8")
        latest_payload = json.loads(payload)
    except json.JSONDecodeError:
        print("Error decoding JSON payload")

# Initialize MQTT client
client_id = f'subscriber_xl_{uuid.uuid4().hex}'
client = mqtt.Client(client_id=client_id, clean_session=False)  # Fixed initialization
client.username_pw_set(username=MQTT_USER, password=MQTT_PWD)

# Assign callbacks
client.on_connect = on_connect
client.on_message = on_message

# Establish connection
def connect_mqtt():
    global broker_connection_status
    while not broker_connection_status:
        try:
            client.connect(MQTT_BROKER, MQTT_PORT)
            client.loop_start()
            time.sleep(2)
        except Exception as e:
            print(f"Connection failed: {e}. Retrying in 5 seconds...")
            time.sleep(5)

# Start MQTT connection in a separate thread
threading.Thread(target=connect_mqtt, daemon=True).start()

# Gradio UI components
def check_status():
    """Returns connection status and latest payload."""
    status = "Connected" if broker_connection_status else "Disconnected"
    color = "green" if broker_connection_status else "red"
    payload = json.dumps(latest_payload, indent=2) if latest_payload else "{}"
    return status, payload

# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# MQTT Status Checker")

    status_button = gr.Button("Disconnected")
    payload_textbox = gr.Textbox(label="Latest Payload", value="", lines=10, interactive=True)

    def update_ui():
        status, payload = check_status()
        return status, payload

    # Auto-update UI every second
    demo.load(update_ui, None, [status_button, payload_textbox], every=1)

# Start the Gradio UI
demo.launch(share=True)


ImportError: cannot import name 'soft_unicode' from 'markupsafe' (/opt/anaconda3/lib/python3.8/site-packages/markupsafe/__init__.py)